In [1]:
#import os
#print(os.environ['PIP_DEFAULT_TIMEOUT'])
## export PIP_DEFAULT_TIMEOUT=100

In [3]:
from math import sqrt, cos, sin, acos, pi, atan2
from matplotlib import cm
from matplotlib import patches
from matplotlib.colors import LogNorm
from matplotlib.path import Path
from mpl_toolkits.mplot3d import Axes3D
from numba import complex128,float64,int64,jit
import matplotlib.pyplot as plt
import numpy as np
import quadpy

#custom module
from fieldplot import GetFlow3D
from nanoscale_test import fieldplot2, rotateAroundX, rotateAroundY, rotateAroundZ, angle2D
from scattnlay import fieldnlay
from scattnlay import scattnlay

import nanoscale_test
import scattnlay

%run kostin_kod.py

In [8]:
from scipy.optimize import differential_evolution
from pyfde import JADE

In [ ]:
r1=120
r2=120

result = differential_evolution(lambda t: -evaluate(t), [(3, r1), (30, r2), (3, 10), (0, 0.4), (3, 10), (0, 0.4),
                                           (0, r2*factor)],
                               popsize=5, maxiter=1000)
print(result)

In [ ]:
r1=120
r2=120

limits = [(10, r1), (30, r2), (3, 10), (0, 0.4), (3, 10), (0, 0.4), (0.1, r2*factor)]

solver = JADE(evaluate, n_dim=len(limits), n_pop=5*len(limits), limits=limits)

best, fit = solver.run(n_it=300)

In [ ]:
def run_evaluation(algo, r_limits, m_limits, mj_limits, layers, quad_n=31):
    limits = [r_limits]*layers+[m_limits,mj_limits]*layers+[(0.1, r_limits[-1]*factor)]
    
    if algo == 'JADE':
        solver = pyfde.JADE(lambda t: evaluate(t, layers=layers, quad_n=quad_n), 
                            n_dim=len(limits), n_pop=5*len(limits), limits=limits)
        best, fit = solver.run(n_it=300)
    elif algo == 'DE':
        result = differential_evolution(lambda t: -evaluate(t, layers=layers, quad_n=quad_n),
                               popsize=5*len(limits), maxiter=1000)
        best, fit = result.x, -result.fun
    else:
        raise AssertionError("Unknown algo")
        
    print("best {} for ind {}".format(fit, best))
    return best, fit

In [6]:
def evaluate(ind, layers=2, quad_n=31):
    assert len(ind)==layers*3+1
    
    r = ind[-1]
    x = ind[:layers]
    m_ = ind[layers:3*layers][::2]
    m_j = ind[layers:3*layers][1::2]
    m = map(lambda t: t[0]+t[1]*1j, zip(m_, m_j))
    
    m = [z for _,z in sorted(zip(x,m), key=lambda pair: pair[0])]
    x = sorted(x)
    
    m = np.array(m)
    x = np.array(x)
    
    coords = get_points('quad', r=r, quad_n=quad_n)
    Iprj = integrand(coords, x, m)
    Ptot = quadpy.sphere.integrate(lambda coords: integrand(coords, x, m),
                            [0.0, 0.0, 0.0], r,
                            quadpy.sphere.Lebedev(str(quad_n))) / (4. * np.pi * r**2)
    return 3./4.*np.max(Iprj)/Ptot

def get_mutate_func(all_mut_pb):
    def mutate(ind):
        new_ind = createIndividual()

        if random.random() < all_mut_pb:
            k = len(ind)
        else:
            k = 1

        for i in random.sample(range(len(ind)), k):
            ind[i] = new_ind[i]

        return ind
    return mutate

In [ ]:
def 